## 2 submissions

In [12]:
import pandas as pd

path1 = '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/1. Learning/submission.csv'
path2 = '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/4. Combined models/submission_M_XG_x10_woDNN.csv'

df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)

# Merge the submissions on isic_id
merged = pd.merge(df1, df2, on="isic_id", suffixes=('_1', '_2'))

### Mean

In [ ]:
# Compute the mean of the two target columns
merged['target'] = merged[['target_1', 'target_2']].mean(axis=1)

# Keep only the required columns
result = merged[['isic_id', 'target']]

# Save to a new CSV
result.to_csv("submission_I_submission_M_XG_x10_woDNN.csv", index=False)

### Highest confidence

In [13]:
# Choose the prediction with stronger confidence (closer to 0 or 1)
merged['target'] = merged.apply(
    lambda row: row['target_1'] if abs(row['target_1'] - 0.5) > abs(row['target_2'] - 0.5) else row['target_2'],
    axis=1
)

# Keep only required columns
result = merged[['isic_id', 'target']]

# Save the result
result.to_csv("submission_I_submission_M_XG_x10_confselect.csv", index=False)

## 3 submissions

In [14]:
import pandas as pd

# Load the three submissions
path1 = '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/1. Learning/submission.csv'
path2 = '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/4. Combined models/submission_M_XG_x10_woDNN.csv'
path3 = '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/4. Combined models/submission_M_RF_x10_woDNN.csv'

df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
df3 = pd.read_csv(path3)

# Merge the three submissions on isic_id
merged = df1.merge(df2, on="isic_id", suffixes=('_1', '_2'))
merged = merged.merge(df3, on="isic_id")
merged.rename(columns={"target": "target_3"}, inplace=True)

In [15]:
# Select the most confident prediction (closest to 0 or 1)
def select_most_confident(row):
    preds = [row['target_1'], row['target_2'], row['target_3']]
    confidences = [abs(p - 0.5) for p in preds]
    return preds[confidences.index(max(confidences))]

merged['target'] = merged.apply(select_most_confident, axis=1)

# Keep only required columns
result = merged[['isic_id', 'target']]

# Save the result
result.to_csv("submission_I_submission_M_XG_x10_I_submission_M_RF_x10_woDNN_confselect.csv", index=False)

## 4 submissions

In [16]:
import pandas as pd

# Load the three submissions
path1 = '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/1. Learning/submission.csv'
path2 = '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/4. Combined models/submission_M_XG_x10_woDNN.csv'
path3 = '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/4. Combined models/submission_M_RF_x10_woDNN.csv'
path4 = '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/2. Image models/submission_I_TunRN50_WRS1000.csv'

df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
df3 = pd.read_csv(path3)
df4 = pd.read_csv(path4)

# Merge all submissions on isic_id
merged = df1.merge(df2, on="isic_id", suffixes=('_1', '_2'))
merged = merged.merge(df3, on="isic_id")
merged.rename(columns={"target": "target_3"}, inplace=True)
merged = merged.merge(df4, on="isic_id")
merged.rename(columns={"target": "target_4"}, inplace=True)

In [17]:
# Select the most confident prediction (closest to 0 or 1)
def select_most_confident(row):
    preds = [row['target_1'], row['target_2'], row['target_3'], row['target_4']]
    confidences = [abs(p - 0.5) for p in preds]
    return preds[confidences.index(max(confidences))]

merged['target'] = merged.apply(select_most_confident, axis=1)

# Keep only required columns
result = merged[['isic_id', 'target']]

# Save the result
result.to_csv("submission_comb4_1.csv", index=False)

## N submissions

In [ ]:
import pandas as pd

def confidence_voting(paths, output_path="submission_confidence_voting.csv"):
    """
    Merges multiple prediction CSVs on 'isic_id' and selects the most confident prediction
    (closest to 0 or 1) for each row.

    Parameters:
        paths (list of str): List of file paths to CSVs containing 'isic_id' and 'target' columns.
        output_path (str): Path to save the final submission CSV.
    """
    if len(paths) < 2:
        raise ValueError("At least two file paths must be provided.")

    # Load and merge all dataframes on 'isic_id'
    dfs = []
    for i, path in enumerate(paths):
        df = pd.read_csv(path).copy()
        df.rename(columns={'target': f'target_{i+1}'}, inplace=True)
        dfs.append(df)

    # Merge all on 'isic_id'
    merged = dfs[0]
    for df in dfs[1:]:
        merged = pd.merge(merged, df, on='isic_id')

    # Select the most confident prediction per row
    target_cols = [f'target_{i+1}' for i in range(len(paths))]
    merged['target'] = merged[target_cols].apply(
        lambda row: row.loc[row.sub(0.5).abs().idxmax()], axis=1
    )

    # Output result
    result = merged[['isic_id', 'target']]
    result.to_csv(output_path, index=False)
    print(f"Saved most confident prediction to: {output_path}")


In [21]:
paths = [
    '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/1. Learning/submission.csv',
    '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/4. Combined models/submission_M_XG_x10_woDNN.csv',
    '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/4. Combined models/submission_M_XG_1.csv',
    '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/4. Combined models/submission_M_RF_x10_woDNN.csv',
    '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/4. Combined models/submission_M_RF_1.csv',
    '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/2. Image models/submission_I_TunRN50_WRS1000.csv',
    '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/2. Image models/submission_I_VGG16_2.csv',
    '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/2. Image models/submission_I_RN50_1.csv',
    '/Users/jorgecamposmarti/Documents/Máster/4o Semicuatrimestre/3. ML (Machine Learning)/Project 2/My Notebooks/4. Combined models/submission_M_XG_x10_finetuned.csv'

]

confidence_voting(paths, output_path="submission_8_models_confident_2.csv")

Saved most confident prediction to: submission_8_models_confident_2.csv
